## IGNORE this, this is a scratchpad for loose cells and exploration of new ideas

In [38]:
num_neg_sentiment_140_rows = sentiment_140_full.target.value_counts()[NEGATIVE_LABEL]
num_sampled_non_neg_sentiment_140_rows = int(
    (non_neg_percent * num_neg_sentiment_140_rows) / 
    (1 - non_neg_percent)
)

In [39]:
sampled_non_neg_sentiment_140 = (
    sentiment_140_full[sentiment_140_full['target'] == POSITIVE_LABEL]
    .sample(num_sampled_non_neg_sentiment_140_rows)
)

In [40]:
sampled_sentiment_140 = pd.concat(
    [
        sentiment_140_full[sentiment_140_full['target'] == NEGATIVE_LABEL],
        sampled_non_neg_sentiment_140
    ]
)

In [41]:
sampled_sentiment_140.target.value_counts()

0    800000
4    477053
Name: target, dtype: int64

In [76]:
sampled_sentiment_140.to_csv('data/distribution_adjusted_sentiment_140.csv', index=False)

# Doubling the original airline data with the sentiment 140 data

In [35]:
num_airline_rows = train.shape[0]
num_neg_airline_rows = train["airline_sentiment"].value_counts()["negative"]
num_non_neg_airline_rows = num_airline_rows - num_neg_airline_rows
non_neg_percent = num_non_neg_airline_rows / num_airline_rows

In [47]:
reduced_sentiment_140 = sampled_sentiment_140.sample(
    n=num_airline_rows, random_state=seed
)
reduced_sentiment_140

,target,text
1149170,4,@tonfue lol haha...so what flavor is the clea...
127890,0,everytime i spend the day in front of the comp...
119417,0,"oh my, the heart got cut off after megan. boo..."
1499133,4,Nicee Ass Day ; Bout To Go Scoop @itsmeambreez...
1167914,4,"@DuckyZu yeah it really is made of awesome, th..."
...,...,...
1371066,4,@veganrunningmom thank you for the FF. i just ...
411933,0,My ugly ID card. Eewwww much? that will be my...
250578,0,@sarahlasvegas http://twitpic.com/6cns4 - no d...
184192,0,Just went to see Drag Me To Hell....very big l...


In [48]:
train

,airline_sentiment,text
0,negative,@united Yes I did. We headed out to de-ice 5 ...
1,negative,@AmericanAir missing a full days of work thank...
2,negative,@USAirways your mobile app is horrible! Needs...
3,negative,@AmericanAir Checked in on app since yesterday...
4,negative,@united we have tried to change our flight THR...
...,...,...
13171,positive,@SouthwestAir replacing @vitaminwater with bee...
13172,negative,@AmericanAir at LAX and your service reps just...
13173,negative,@SouthwestAir Been on hold for over an hour - ...
13174,negative,@united we would...how do I contact you to dis...


Definitely noticing a difference in the tone of the sentiment 140 tweets compared to the airline tweets

In [53]:
airline_full_props = value_count_proportions(train['airline_sentiment'])

In [52]:
value_count_proportions(reduced_sentiment_140['target'])

{0: 0.6218123861566485, 4: 0.37818761384335153}

We want ratio of negative:positive to be equal in both the airline data and the target data.

In [55]:
pos_and_neg = ['positive', 'negative']
positive_negative_prop_sum = sum(airline_full_props[sent] for sent in pos_and_neg)
airline_positive_negative_props = {
    sent: airline_full_props[sent] / positive_negative_prop_sum
    for sent in pos_and_neg
}
airline_positive_negative_props

{'positive': 0.2047403410733211, 'negative': 0.7952596589266788}

## Loading data from [tweet sentiment support phrase data](https://www.kaggle.com/c/tweet-sentiment-extraction)

This dataset actually does have neutral tweets. I want to augment the airline data with this new dataset while minimizing the number of rows lost from the new tweet support phrase dataset.

In [78]:
support_phrase_data = pd.read_csv('data/support_phrase_data.csv')

In [91]:
support_phrase_data['sentiment'].value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [95]:
negative_rows = support_phrase_data['sentiment'].value_counts()['negative']

In [89]:
sentiment_proportions = value_count_proportions(support_phrase_data['sentiment'])

In [90]:
sentiment_proportions

{'neutral': 0.4045704304792402,
 'positive': 0.3122884902296132,
 'negative': 0.2831410792911466}

This is different than the airline sentiment proportions which were
```
{'negative': 0.6264420157862781,
 'neutral': 0.21227990285367335,
 'positive': 0.16127808136004856}
```
We know the desired proportions of neutral and positive rows. We know that we don't want to shed any negative rows (since they were originally the majority class, we want to keep as many of them around now that they're the minority class).
We have to remove some positive and neutral rows. How many? This sounds like a [system of equations](https://www.wolframalpha.com/input/?i=n+%2F+%287781+%2B+n+%2B+p%29+%3D+0.21%2C+p+%2F+%287781+%2B+n+%2B+p%29+%3D+0.17). `n` is the desired number of neutral rows, `p` is the desired number of positive rows, and `7781` is the number of negative rows in the support phrase data.
We get:
- num positive rows: `2636`
- num neutral rows: `2133`

In [94]:
import sympy as sym
sym.init_printing()

In [99]:
pos_rows, neutral_rows = sym.symbols('p,n')
neutral_eq = sym.Eq(
    neutral_rows / (negative_rows + pos_rows + neutral_rows),
    airline_sent_proportions['neutral']
)
positive_eq = sym.Eq(
    pos_rows / (negative_rows + pos_rows + neutral_rows),
    airline_sent_proportions['positive']
)

sym.solve([neutral_eq, positive_eq], [pos_rows, neutral_rows])

In [2]:
t = TextClasDataBunch.from_csv('data', 'original_train.csv', bs=32, text_cols='text', label_cols='airline_sentiment')

NameError: name 'TextClasDataBunch' is not defined

In [3]:
filename = 'data/distribution_adjusted_sentiment_140.csv'
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)

In [5]:
s = 30000
skiprows = sorted(random.sample(range(1,n+1),n-s))

In [11]:
df = pd.read_csv(filename, skiprows=skiprows)

In [17]:
df['sentiment'] = df['sentiment'].apply(
    lambda x: 'negative' if x == 0 else 'positive'
)

In [19]:
df.to_csv('data/trimmed_sentiment_140.csv', index=False)

TODO: make sure to be clear that you distibution adjusted AND sampled the sentiment 140 to have a 3:1 ratio to the airline data

In [4]:
test = pd.read_csv('data/original_test.csv')

In [8]:
value_count_proportions(test.airline_sentiment)

{'negative': 0.6311475409836066,
 'neutral': 0.2062841530054645,
 'positive': 0.16256830601092895}